In [ ]:
import time
import easygopigo3 as easy
import numpy as np

gpg = easy.EasyGoPiGo3()
actions = []

#we split the command (each vector of the list) in the Name and the Parameters
def parse_command(command):
    tokens = command.split()
    structured_command = {}
    name = tokens[0]
    parameters = tokens[1::]
    structured_command["NAME"] = name
    structured_command["PARAMETERS"] = parameters
    return structured_command

#we check that the name exists, that the dimension of the parameters match and that the specified parameters are correct for each case
def is_valid_command(command):
    is_valid = False
    if structured_command["NAME"] != "SET" and structured_command["NAME"] != "STOP "and structured_command["NAME"] != "MV":
            is_valid = False
    if structured_command["NAME"] == "SET":
        if len(structured_command["PARAMETERS"]) != 2:
            is_valid = False
        else:
            is_valid = len(structured_command["PARAMETERS"]) == 2
            is_valid = is_valid and structured_command["PARAMETERS"][0].upper() == "SPEED"
            is_valid = is_valid and structured_command["PARAMETERS"][1].isnumeric()
            is_valid = is_valid and int(structured_command["PARAMETERS"][1]) < 255
            is_valid = is_valid and int(structured_command["PARAMETERS"][1]) > 0
    
    if structured_command["NAME"] == "MV":
        if len(structured_command["PARAMETERS"]) != 1:
            is_valid = False
        else:
            is_valid = structured_command["PARAMETERS"][0].upper() == "R" or structured_command["PARAMETERS"][0].upper() == "L" or structured_command["PARAMETERS"][0].upper() == "B" or structured_command["PARAMETERS"][0].upper() == "F"
    if structured_command["NAME"] == "STOP":
        if len(structured_command["PARAMETERS"]) != 0:
            is_valid = False
        else:
            is_valid = True
    return is_valid

#we execute the orders calling functions for the robot (gpg.function())
def execute_command(command):
    structured_command = parse_command(command)
    if structured_command["NAME"] == "SET":
        i = int(structured_command["PARAMETERS"][1])
        gpg.set_speed(i)
    if structured_command["NAME"] == "STOP":
        gpg.stop()
    if structured_command["NAME"] == "MV":
        actions.append("MV"+str(structured_command["PARAMETERS"]))
        if structured_command["PARAMETERS"][0] == "R":
            gpg.turn_degrees(90)
            gpg.forward()
        if structured_command["PARAMETERS"][0] == "L":
            gpg.turn_degrees(-90)
            gpg.forward()
        if structured_command["PARAMETERS"][0] == "F":
            gpg.forward()
        if structured_command["PARAMETERS"][0] == "B":
            gpg.turn_degrees(180)
            gpg.forward()

    actions.append(structured_command)

if __name__=="__main__":
    #we ask for the name file, open it and append each of its lines (with .strip() and .upper()) to a list (commands)
    name = input("Tell me the file name (without the .txt): \n").upper()
    name += ".txt"
    f = open(name, "r")
    lines = f.readlines()
    commands = []
    for i in range(len(lines)):
        commands.append((lines[i].strip()).upper())
    #we append an Stop at the end, so the program finishes once all the orders where done
    commands.append("STOP")
    #print(commands)
    f.close()
    #we specify how much time we wanto the robot to be doing the two possible tasks: move once set a direction or move once set a velocity
    print("Specify the time parameters: ")
    t_direction = int(input("Time moving in the direction given in ms (once is set to that direction): "))
    t_speed = int(input("Time moving when set new speed in ms (the setting is inmediate): "))

    #we make sure that every line is correct
    is_all_valid = True
    for command in commands:
        structured_command = parse_command(command)
        is_all_valid = is_all_valid and is_valid_command(structured_command)
    print(is_all_valid)
    #if everything is correct we execute the orders
    if is_all_valid:
        for command in commands:
            print(command)
            execute_command(command)
            structured_command = parse_command(command)
            if structured_command['NAME'] == "SET":
                time.sleep(t_speed/1000)
            if structured_command['NAME'] == "MV":
                time.sleep(t_direction/1000)
    #in the case that same line has a command wrong written, (or impossible), then this messages shall be displayed
    else:
        print("One of the actions specified can not be done. Check all the expression, here are the things that can be written (each of each line): ")
        print("For moving: MV and the direction (L, R, F, B)")
        print("For setting the speed: SET SPEED")
        print("For stopping the motion: STOP")
        print("Take into account that once stopped, you can done anything else.")
      
